# DTSC 680: Applied Machine Learning
# Name: Juliana Meirelles
# Assignment 6: Mushroom Classifier

Overview:

The purpose of this assignment was to gain experience using KNN and PCA
algorithms using the Mushroom Data set. This dataset is part of the UCI Machine Learning Repository. This data set includes many different features that categorize mushrooms by colors, shapes, etc. All of these features are used to classify mushrooms by either edible or poisonous. This is very helpful for people who enjoy mushroom hunting or foraging for mushrooms as many mushrooms are edible! Many mushrooms, though, are poisonous so mycologists have determined certain aspects that help to identify if a mushroom is edible or not.

My process for this project:
1. Import the Mushroom data set and view the data to understand what is going on.
2. Use the KNN algorithm to impute missing values in the dataset. To do this, I will split the data into four subsets that will serve as training and testing data. This will be split according to data that is missing from one column in the data set. 
3. Train a RandomForestClassifier as well as a LogisticRegression model to predict whether a mushroom is edible or poisonous given this data set.
4. Compute the accuracy, precision, and recall scores for a test set.
5. Perform dimensionality reduction using PCA and keep 95% of the variance.
6. Train two new models, a RandomForestClassifier and a LogisticRegression model, on this reduced dataset to predict whether a mushroom is edible or not.
7. Compute the accuracy, precision, and recall scores for the model trained on the reduced data set using the same test set as before.

## Preliminaries
Importing Commonly used Packages

In [1]:
# Common imports
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import os

## Import the Data

In [2]:
#Here, I am importing the Mushroom Data Set. Also I am renaming the columns because I have no idea what these one 
#letter columns mean and I need some reference points as to what features I'm working with.

data = pd.read_csv("agaricus-lepiota.data", names = ['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 
                                                       'odor', 'gill-attachment','gill-spacing', 'gill-size', 
                                                       'gill-color', 'stalk-shape', 'stalk-root',
                                                       'stalk-surface-above-ring', 'stalk-surface-below-ring', 
                                                       'stalk-color-above-ring','stalk-color-below-ring', 
                                                       'veil-type', 'veil-color', 'ring-number', 'ring-type',
                                                       'spore-print-color', 'population', 'habitat'])

pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

mushroom_df = pd.DataFrame(data)

In [4]:
#Viewing my data 

mushroom_df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


In [5]:
#Viewing my data, understanding the different features and columns

mushroom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [6]:
#Still viewing my data to understand the diferent types of data recorded here

mushroom_df.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,2,5,4,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,t,b,s,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,4608,3776,5176,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [7]:
#Viewing my data. This tells me how many mushrooms are classified as edible and how many are classified as poisonous

mushroom_df['class'].value_counts()

e    4208
p    3916
Name: class, dtype: int64

## Use the KNN algorithm to impute missing values in the dataset

In [6]:
#Creating features dataframe. This includes all columns except stalk root

features = mushroom_df[['class','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment',
                       'gill-spacing','gill-size','gill-color','stalk-shape','stalk-surface-above-ring',
                       'stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type', 
                       'veil-color','ring-number','ring-type','spore-print-color','population','habitat']]

In [7]:
#Creating response dataframe. This includes only the stalk root column that has missing values

response = mushroom_df[['stalk-root']]

In [8]:
#One-Hot-Encoding Feature data per the instructions of the project. This will turn the features into just numbers
#so that the machine learning algorithms can understand them

from sklearn.preprocessing import OneHotEncoder

features_encode = OneHotEncoder(sparse = False)

features = features_encode.fit_transform(features)

In [9]:
#Here, I am creating a mask so that I can create training and testing sets based off of whether or not there is are 
#missing values. My mask will make a missing value be True, and a non-missing value False. Then I implement my mask

mask = mushroom_df['stalk-root'] == '?'

In [12]:
#Creating my training data using my mask from above. The 'false' means that I am only pulling non missing values,
#since my mask made True = ?

training = features[mask == False]

In [13]:
#Creating my testing data using my mask from above. This pulls my missing values

testing = response[mask]

In [14]:
train_f = features[mask == False] #Training features with no missing values

train_r = response[~mask] #Training responses where there are missing values

test_f = features[mask] #Testing features where there are missing values

test_r = response[mask] #Testing resposes which is our list of ?s. This will be used after prediction

In [15]:
#Label encoding the response data for training only. 

from sklearn import preprocessing

label_encode = preprocessing.LabelEncoder()

train_r = label_encode.fit_transform(train_r)

/Users/julianam/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [16]:
#Implementing my KNN classifier model and fitting it with my training features and responses. In the last line, 
#I predict the missing values by passing it the test features. This will replace the original missing responses

from sklearn.neighbors import KNeighborsClassifier

#Default Settings
imputer = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30,
                               p=2, metric='minkowski', metric_params=None, n_jobs=None)

imputer.fit(train_f, train_r)

missing_values = imputer.predict(test_f)

In [17]:
#Create data structure (list) called missing_values that contains all of the imputed values 

missing_values = label_encode.inverse_transform(missing_values)
missing_values

array(['b', 'b', 'e', ..., 'e', 'b', 'e'], dtype=object)

In [18]:
#Replacing the ? mark list that is all missing values with my predicted values list from above

test_r['stalk-root'] = missing_values

<ipython-input-18-d1852d8ceb2d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_r['stalk-root'] = missing_values


In [19]:
#Here, I am inserting the predicted values into the original dataframe so I can move forward on this assignment 
#with a full dataset and no missing data

mushroom_df[mask] = test_r

## Graded Concept Question #1: Would it still be possible to train the KNN model if you one-hot encoded the response data instead? Why or why not?

Yes, it would still be possible to train the KNN model if you one-hot encoded the response data instead of label encoding the response data because the response data is not ordinal. One-hot encoding the data would make it binary, either 0 or 1 depending on if it was a missing value or not. There is no assigned order with One-Hot encoding, ex: 1 < 2 < 3. The label encoding makes the data ordinal, so the computer interprets values as being ranked. Label encoding works well for storing the values with less space being used up. One-hot encoding can become difficult in terms of dimensionality so it is used in conjunction with PCA. 

In [20]:
#Creating my X and y data. X are all the features that predict the class. Y is class because that is what we are
#aiming to predict (edible or poisonous)

X = mushroom_df[['cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment',
                       'gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring',
                       'stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type', 
                       'veil-color','ring-number','ring-type','spore-print-color','population','habitat']]

y = mushroom_df[['class']]

In [21]:
#One hot encode X as per instructions (feautres)

OH_encoder = OneHotEncoder(sparse = False)

X = OH_encoder.fit_transform(X)

In [24]:
#Label encode y as per instructions (responses)

label_encoder = preprocessing.LabelEncoder()

y = y.ravel()

y = label_encoder.fit_transform(y)

In [25]:
#Using good 'ole train-test split by sklearn to do a random split of the data and create training sets and testing
#sets. The testing sets will be used to see the accuracy of our models. 

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## Train a RandomForestClassifier as well as a LogisticRegression model to predict whether a mushroom is edible or poisonous given this data set of nominally-valued characteristics.

In [26]:
#Instatiating an object of the RandomForestclassifier class and training it on the training data

from sklearn.ensemble import RandomForestClassifier

rft_clf = RandomForestClassifier(random_state = 42)

rft_clf.fit(X_train,y_train)

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


In [27]:
#Instatiating an object of the LogisticRegression class and training it on the training data

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state = 42)

log_reg.fit(X_train, y_train)

%time

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 5.01 µs


## Graded Concept Question #2: Would it still be possible to train these two models if you one-hot encoded the response data instead, being careful to specify that the drop parameter of the OneHotEncoder class is set to ‘first’? Why or why not?

Yes, you would be able to one hot encode the response data instead of label encoding the response data because the response data does not have an order to it. It is either edible or poisonous. One is not worth more than the other.

## Computing the accuracy, precision, and recall scores for a test set on both the RandomForestClassifer model and the Logistic Regression model

In [28]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


rft_predict = rft_clf.predict(X_test)
log_predict = log_reg.predict(X_test)

#RandomForestClassifier
rft_acc_score = accuracy_score(y_test, rft_predict)
rft_prec_score = precision_score(y_test, rft_predict, average = 'micro') 
rft_recall_score = recall_score(y_test, rft_predict, average = 'micro')

print('Random Forest Accuracy = %s' % (rft_acc_score))
print('Random Forest Precision = %s' % (rft_prec_score))
print('andom Forest Recall = %s' % (rft_recall_score))


#LogisticRegression
log_acc_score = accuracy_score(y_test, log_predict)
log_prec_score = precision_score(y_test, log_predict, average = 'micro') 
log_recall_score = recall_score(y_test, log_predict, average = 'micro')

print('Logistic Regression Accuracy = %s' % (log_acc_score))
print('Logistic Regression Precision = %s' % (log_prec_score))
print('Logistic Regression Recall = %s' % (log_recall_score))

Random Forest Accuracy = 1.0
Random Forest Precision = 1.0
andom Forest Recall = 1.0
Logistic Regression Accuracy = 1.0
Logistic Regression Precision = 1.0
Logistic Regression Recall = 1.0


## Quick Analysis

With the data we were provided, overfitting is clear here. We can see this through the accuracy scores for both the Random Forest Classifier and the Logistic Regression models. An accuracy score of 1 is not normal and is not what we want to see because the model is too good at predicting data it has seen before and will struggle to predict data it has not seen before. 

## Performing dimensionality reduction using PCA and keep 95% of the variance

In [29]:
from sklearn.decomposition import PCA

#Here, I am performing dimensionality reduction using PCA and I am specifying in the n_components parameter that
#I want to keep 95% of the variance as specified in the project instructions. I can do this by using a float number
#between 0.0 and 1.0. 

pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

#Here, I am printing out the original number of dimensions in our X dataframe. Then I print out the number of
#dimensions after performing dimensionality reduction. Lastly, I print out the percent that I was able to reduce
#the number of dimensions by 

full_dims = X.shape[1]
reduced_dims = X_reduced.shape[1]

print('Original number of dimensions: ',full_dims)
print('Number of dimensions after PCA: ',reduced_dims)
print('Percentage that I was able to reduce dimensions by: ',round((full_dims-reduced_dims)/full_dims*100,2),'%')

Original number of dimensions:  146
Number of dimensions after PCA:  29
Percentage that I was able to reduce dimensions by:  80.14 %


## Training two new models, a RandomForestClassifier and a LogisticRegression model, on the reduced dataset to predict whether a mushroom is edible or not.

In [30]:
#Doing a new train test split using the reduced data

X_train_reduced, X_test_reduced, y_train, y_test = train_test_split(X_reduced, y, test_size=0.20, random_state=42)

In [31]:
from sklearn.ensemble import RandomForestClassifier

rft_clf_reduced = RandomForestClassifier(random_state = 42)

rft_clf_reduced.fit(X_train_reduced,y_train)

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [32]:
from sklearn.linear_model import LogisticRegression

log_reg_reduced = LogisticRegression()

log_reg_reduced.fit(X_train_reduced, y_train)

%time

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 4.05 µs


## Computing the accuracy, precision, and recall scores for the model trained on the reduced data set using the same test set as before.

In [33]:
#Making my predictions on my newly trained reduced models

rft_red_predict = rft_clf_reduced.predict(X_test_reduced)
log_red_predict = log_reg_reduced.predict(X_test_reduced)

In [34]:
#RandomForestClassifier Accuracy, Precision, and Recall Scores

rft_reduced_acc_score = accuracy_score(y_test, rft_red_predict)
rft_reduced_prec_score = precision_score(y_test, rft_red_predict, average = 'micro') 
rft_reduced_recall_score = recall_score(y_test, rft_red_predict, average = 'micro')

print('Reduced Random Forect Classifier Accuracy = %s' % (rft_reduced_acc_score))
print('Reduced Random Forect Classifier Precision = %s' % (rft_reduced_prec_score))
print('Reduced Random Forect Classifier Recall = %s' % (rft_reduced_recall_score))


#LogisticRegression Accuracy, Precision, and Recall Scores

log_reduced_acc_score = accuracy_score(y_test, log_red_predict)
log_reduced_prec_score = precision_score(y_test, log_red_predict, average = 'micro') 
log_reduced_recall_score = recall_score(y_test, log_red_predict, average = 'micro')

print('Reduced Logistic Regression Accuracy = %s' % (log_reduced_acc_score))
print('Reduced Logistic Regression Precision = %s' % (log_reduced_prec_score))
print('Reduced Logistic Regression Recall = %s' % (log_reduced_recall_score))

Reduced Random Forect Classifier Accuracy = 1.0
Reduced Random Forect Classifier Precision = 1.0
Reduced Random Forect Classifier Recall = 1.0
Reduced Logistic Regression Accuracy = 0.9987692307692307
Reduced Logistic Regression Precision = 0.9987692307692307
Reduced Logistic Regression Recall = 0.9987692307692307


## Tabulating the model information

Models              | Item      | Full Data | PCA Reduced |
|------------------:|:---------:|:----------|:------------|
|Random Forest      |Accuracy   | 1.0       | 1.0         |
|    -              | Precision | 1.0       | 1.0         |
|    -              | Recall    | 1.0       | 1.0         |
|    -              | Time      |2 µs       | 2 µs        |
|Logistic Regression| Accuracy  | 1.0       | 0.9987      |
|    -              | Precision | 1.0       | 0.9987      |
|    -              | Recall    | 1.0       | 0.9987      |
|    -              | Time      |5 µs       | 5 µs        |

## Conclusions can you make about these models and the PCA process? How do the models compare? Discuss the trends observed for model training time and performance for the full and reduced data sets.

The PCA process reduced the dimensionality of the data and improved the accuracy, precision, and recall scores for the Logistic Regression model. It did not improve the scores for my Random Forest model. Therefore, PCA improved the performance of the models when compared to the full data set.